<img src="https://github.com/microsoft/Reporting-Services/blob/master/Docs/SSRS.PNG?raw=true" width="10%">
<img src="https://github.com/Microsoft/azuredatastudio/blob/master/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="10%">

# This PowerShell Notebook will walk you through how to download the SQL Performance Dashboard and deploy it to an SSRS server.

If the ReportingServicesTools is not present, download the ReportingServicesTools module from GitHib

In [2]:
try {Import-Module ReportingServicesTools -ErrorAction Stop} catch {Install-Module ReportingServicesTools -Scope CurrentUser } finally {Import-Module ReportingServicesTools}

Setting our GitHub resources to variables, downloading, and extracting the files

In [ ]:
$ZipURL = "https://github.com/Microsoft/tigertoolbox/raw/master/SQL-performance-dashboard-reports/SQL%20Server%20Performance%20Dashboard%20Reporting%20Solution.zip"
$SQLURL = 'https://github.com/Microsoft/tigertoolbox/raw/master/SQL-performance-dashboard-reports/setup.sql'

<# Set destination file locations #>
$ZipFile = "$($env:USERPROFILE)\Downloads\SQLServerPerformanceDashboardReportingSolution.zip"
$ReportsBaseFolder = 'C:\SQL Server Performance Dashboard'
$SQLFile = "$($ReportsBaseFolder)\Setup.SQL"

<# Start up a web client and download the .zip fole from GitHub #>
$webclient = New-Object system.net.webclient
$webclient.DownloadFile($ZipURL,$ZipFile)
<# UnZip the Reporting Solution Zip file #>
Expand-Archive $ZipFile -DestinationPath $ReportsBaseFolder

<# Now that the reports are unzipped, download the SQL file to that same directory #>
$webclient.DownloadFile($SQLURL,$SQLFile)

Deploying the reports to our SSRS or Power BI Reports Server

In [ ]:
<# SSRS Instance, this is where the reports will be rendered from.
    You probably need to change this to something like 'http://MyReportServer/ReportServer'.  If you have a named SSRS instance 'http://MyReportServer/ReportServer_SQL2016'  #>
$SSRSInstance = 'http://localhost/ReportServer'
$NewSSRSReportFolder = 'SQL Server Performance Dashboard'

<# Deploy the dashboard reports to the $NewSSRSReportFolder ('SQL Server Performance Dashboard') folder of an SSRS instance
    Note: We are creating the folder using New-RsFolder, you may need to skip this step if you’ve already run it once. #>
New-RsFolder -ReportServerUri $SSRSInstance -Path / -Name $NewSSRSReportFolder

Write-RsFolderContent -ReportServerUri $SSRSInstance -Path "$($ReportsBaseFolder)\SQL Server Performance Dashboard\" -Destination /$NewSSRSReportFolder
    

Deploy the required SQL Queries to each instance of SQL Server you want to monitor.

In [ ]:
foreach ($RegisteredSQLs IN dir -recurse SQLSERVER:\SQLRegistration\'Database Engine Server Group'\ | where {$_.Mode -ne 'd'} )
{
Invoke-Sqlcmd -ServerInstance $RegisteredSQLs.Name -Database msdb -InputFile $SQLFile
}

Now go to your SSRS server and have a look at your SQL Performance Dashboard.

In [ ]:
<# Go to SSRS and make sure everythng works
    You URL should look something like http://localhost/Reports/report/SQL%20Server%20Performance%20Dashboard/performance_dashboard_main
    Continur with Setp #8 back at https://blogs.msdn.microsoft.com/sql_server_team/sql-server-performance-dashboard-reports-unleashed-for-enterprise-monitoring/  #>
Start-Process "$($SSRSInstance -replace 'Server', 's')/report/$($NewSSRSReportFolder)/performance_dashboard_main"